# Machine Learning Assignment 2

Daniel Devlin | 19202103 | daniel.devlin1@ucdconnect.ie

#### Imports

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'tensorflow'

#### Load Student_ID Dataset

In [3]:
census_dataset = 'census_19202103.csv'
df_census = pd.read_csv(census_dataset)
df_census

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
0,49,Private,151107,HS-grad,9,Never-married,Craft-repair,Not-in-family,Black,Male,0,0,40,United-States,<=50K
1,49,Private,248895,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,45,United-States,<=50K
2,60,Private,227266,HS-grad,9,Divorced,Sales,Not-in-family,White,Female,0,0,33,United-States,<=50K
3,62,Self-emp-not-inc,192236,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,>50K
4,23,Private,386019,9th,5,Never-married,Farming-fishing,Unmarried,White,Male,0,0,70,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40,Private,256202,Assoc-voc,11,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
9996,34,Private,100669,Bachelors,13,Married-civ-spouse,Transport-moving,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>50K
9997,74,Federal-gov,39890,Some-college,10,Widowed,Transport-moving,Not-in-family,White,Female,0,0,18,United-States,<=50K
9998,32,Private,193042,Prof-school,15,Married-civ-spouse,Sales,Husband,White,Male,15024,0,60,United-States,>50K


## [_] Question 1

### [_] (a) Clean and Prepare Dataset for ML Analysis

#### Checking for Issues in the Dataframe

In [4]:
# Check for Missing Values
missing = df_census.isnull().sum(axis=1)
rows_with_missing_values = (missing > 0).sum()
print("Total of: %d rows with missing values" % (rows_with_missing_values))
print("\nTotal of: %d missing values in the dataset" % (missing.sum()))

Total of: 0 rows with missing values

Total of: 0 missing values in the dataset


No rows with missing values (as of yet lol). 

In [5]:
# Checking For Duplicates
df_census[df_census.duplicated()]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
8947,20,Private,107658,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,10,United-States,<=50K


One duplicate row identified!

In [6]:
# Checking Column Titles for Whitespace
df_census.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' Income-category'],
      dtype='object')

All columns but age have a whitespace first before text. This can lead to errors in readability and column access. 

In [7]:
#Checking Column datatypes
df_census.dtypes

age                  int64
 workclass          object
 fnlwgt              int64
 education          object
 education-num       int64
 marital-status     object
 occupation         object
 relationship       object
 race               object
 sex                object
 capital-gain        int64
 capital-loss        int64
 hours-per-week      int64
 native-country     object
 Income-category    object
dtype: object

In [8]:
#Examining an object column's values
df_census[' occupation'].unique()

array([' Craft-repair', ' Machine-op-inspct', ' Sales', ' Prof-specialty',
       ' Farming-fishing', ' Handlers-cleaners', ' ?', ' Exec-managerial',
       ' Protective-serv', ' Adm-clerical', ' Tech-support',
       ' Other-service', ' Transport-moving', ' Priv-house-serv',
       ' Armed-Forces'], dtype=object)

Despite noticing that there is also whitespace in front of values in these rows, there is a value of '?' that obviously mean it is a missing value. 

#### Fixing First Found Issues

In [9]:
#Removing Duplicates
df_census = df_census.drop_duplicates()
len(df_census)

9999

In [10]:
#Removing Column Whitespaces
df_census.columns = df_census.columns.str.lstrip() 
df_census.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Income-category'],
      dtype='object')

#### Further Issues

In [11]:
# Change income-category to 0 or 1
df_census['Income-category'].unique()

array([' <=50K', ' >50K'], dtype=object)

The target variable of this dataset for our machine learning analysis in this assignment is the income-category column. There are two values (with a whitespace before) in this column amongst the rows, <=50K and >50K. In order to easier plot and access these values for analysis, changing them to a binary 0 or 1 value, representing if a subject's income is under or over 50K is easier than the object values of the original. 

In [12]:
#Create Binary Values in Income-Category column
col_vals = {' <=50K': 0, ' >50K': 1}
df_census['Income-category'] = df_census['Income-category'].map(col_vals)
df_census.head()

/tmp/ipykernel_3561/4232361688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_census['Income-category'] = df_census['Income-category'].map(col_vals)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
0,49,Private,151107,HS-grad,9,Never-married,Craft-repair,Not-in-family,Black,Male,0,0,40,United-States,0
1,49,Private,248895,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,45,United-States,0
2,60,Private,227266,HS-grad,9,Divorced,Sales,Not-in-family,White,Female,0,0,33,United-States,0
3,62,Self-emp-not-inc,192236,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,1
4,23,Private,386019,9th,5,Never-married,Farming-fishing,Unmarried,White,Male,0,0,70,United-States,0


In [13]:
#Remove Whitespace from object columns
df_census = df_census.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_census

/tmp/ipykernel_3561/3218866406.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_census = df_census.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
0,49,Private,151107,HS-grad,9,Never-married,Craft-repair,Not-in-family,Black,Male,0,0,40,United-States,0
1,49,Private,248895,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,45,United-States,0
2,60,Private,227266,HS-grad,9,Divorced,Sales,Not-in-family,White,Female,0,0,33,United-States,0
3,62,Self-emp-not-inc,192236,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,1
4,23,Private,386019,9th,5,Never-married,Farming-fishing,Unmarried,White,Male,0,0,70,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40,Private,256202,Assoc-voc,11,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
9996,34,Private,100669,Bachelors,13,Married-civ-spouse,Transport-moving,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
9997,74,Federal-gov,39890,Some-college,10,Widowed,Transport-moving,Not-in-family,White,Female,0,0,18,United-States,0
9998,32,Private,193042,Prof-school,15,Married-civ-spouse,Sales,Husband,White,Male,15024,0,60,United-States,1


In [14]:
# Replace Question Marks with NaN values
df_census = df_census.replace('?', np.nan)
df_census['occupation'].unique()

array(['Craft-repair', 'Machine-op-inspct', 'Sales', 'Prof-specialty',
       'Farming-fishing', 'Handlers-cleaners', nan, 'Exec-managerial',
       'Protective-serv', 'Adm-clerical', 'Tech-support', 'Other-service',
       'Transport-moving', 'Priv-house-serv', 'Armed-Forces'],
      dtype=object)

In [15]:
# Remove rows with Missing Values(NaN)
nan_rows = df_census.isna().any(axis=1)
nan_rows = nan_rows.sum()
print(nan_rows)

721


In [16]:
# Create dataframe with no missing values
df_census_NONAN = df_census.dropna()
df_census_NONAN

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income-category
0,49,Private,151107,HS-grad,9,Never-married,Craft-repair,Not-in-family,Black,Male,0,0,40,United-States,0
1,49,Private,248895,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,45,United-States,0
2,60,Private,227266,HS-grad,9,Divorced,Sales,Not-in-family,White,Female,0,0,33,United-States,0
3,62,Self-emp-not-inc,192236,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,1
4,23,Private,386019,9th,5,Never-married,Farming-fishing,Unmarried,White,Male,0,0,70,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40,Private,256202,Assoc-voc,11,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
9996,34,Private,100669,Bachelors,13,Married-civ-spouse,Transport-moving,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
9997,74,Federal-gov,39890,Some-college,10,Widowed,Transport-moving,Not-in-family,White,Female,0,0,18,United-States,0
9998,32,Private,193042,Prof-school,15,Married-civ-spouse,Sales,Husband,White,Male,15024,0,60,United-States,1


In [17]:
#Save Dataframes to CSV's for ease
output_folder = './Dataframes/'
df_census.to_csv(output_folder + 'df_census.csv', index=False)
df_census_NONAN.to_csv(output_folder + 'df_census_NONAN.csv', index=False)

#### (a) Discussion: Why did I use these evaluation measures and what was cleaned in the dataframe? 


After code and visual analysis of the pandas dataframe, I identified the following issues/potential issues with the census dataframe loaded in: one duplicate row, values of '?' in some object columns identifying missing values, whitespaces in front of some column values and row values, and a potential confusing values in the target column(Income-category).

These issues were addressed and removed/changed. We are left with two dataframes: df_census: a dataframe containing rows with at most one missing value in an object column, and df_census_NONAN: the same dataframe with 721 rows containing misisng values removed. 


### (b) Evaluate Performance of 3 Basic Classifiers

In [18]:
#Load previous dataframes
df_census = pd.read_csv('./Dataframes/df_census.csv')
df_census_NONAN = pd.read_csv('./Dataframes/df_census_NONAN.csv')

#### Splitting into Test and Train Sets

In [26]:
df_encoded = pd.get_dummies(df_census_NONAN, columns=['education', 'workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
df_encoded

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,Income-category,education_10th,education_11th,education_12th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,49,151107,9,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,49,248895,9,0,0,45,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,60,227266,9,0,0,33,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,62,192236,13,0,0,40,1,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,23,386019,5,0,0,70,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,40,256202,11,0,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9274,34,100669,13,0,0,40,1,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9275,74,39890,10,0,0,18,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
9276,32,193042,15,15024,0,60,1,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [29]:
x = df_encoded.drop(columns = ['Income-category']) #Setting x as the dataframe without our target feature
y = df_encoded['Income-category'] #Setting y to our target feature
y = pd.DataFrame(y) #Creating a dataframe from that Pandas series
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42) #Splitting into training and test sets

#### Decision Tree with Depth at Most 3

In [30]:
dtc_1 = DecisionTreeClassifier(criterion='entropy', max_depth=3) #Creating a Decision Tree Classifier as dtc_1 with entropy and a max_depth of 3
dtc_1.fit(X_train, y_train) #Feeding our Training sets to dtc_1

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [31]:
dtc_y_predicted_train = dtc_1.predict(X_train) #Calculating a prediction of the training set
dtc_train_f1 = f1_score(y_train, dtc_y_predicted_train) #Finding F Score
dtc_train_f1

0.6071428571428571

In [32]:
dtc_y_predicted_test = dtc_1.predict(X_test)
dtc_test_f1 = f1_score(y_test, dtc_y_predicted_test)
dtc_test_f1

0.5865284974093264

#### Neural Network with at Most 10 Nodes

In [33]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(preprocessor.fit_transform(X_train), y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(preprocessor.transform(X_test), y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

NameError: name 'keras' is not defined

#### 1-NN

#### (b) Discussion of Classifiers

Text

### (c)

### (d)

### (e)

## Question 2

### (a)

### (b)

### (c)

### (d)